In [1]:
import numpy as np
import cv2

class VisionUtils:
    """
    Utility class for Week 1 Computer Vision foundations: 
    Linear Algebra, Camera Projection, and Triangulation.
    """

    @staticmethod
    def get_projection_matrix(K, R, t):
        """
        Computes the 3x4 projection matrix P = K @ [R | t].
       
        """
        extrinsic = np.hstack([R, t.reshape(3, 1)])
        P = K @ extrinsic
        return P

    @staticmethod
    def project_3d_to_2d(P, X_3d):
        """
        Projects a 3D point (or array of points) to 2D image coordinates.
        X_3d: (N, 3) or (3,) array
       
        """
        X_3d = np.atleast_2d(X_3d)
        # Convert to homogeneous coordinates [x, y, z, 1]
        X_homo = np.hstack([X_3d, np.ones((X_3d.shape[0], 1))])
        
        # Project: x = P @ X
        x_2d_homo = (P @ X_homo.T).T
        
        # Normalize by the last coordinate (depth)
        x_2d = x_2d_homo[:, :2] / x_2d_homo[:, 2:3]
        return x_2d.squeeze()

    @staticmethod
    def check_epipolar_constraint(p1, p2, F):
        """
        Validates the epipolar constraint: p2^T @ F @ p1 ≈ 0.
       
        """
        # Convert points to homogeneous coordinates if necessary
        p1_h = np.append(p1, 1)
        p2_h = np.append(p2, 1)
        
        error = p2_h.T @ F @ p1_h
        return error

    @staticmethod
    def triangulate_dlt(p1, p2, P1, P2):
        """
        Direct Linear Transform (DLT) triangulation.
        Finds the 3D point X that minimizes the error for p1 ≈ P1@X and p2 ≈ P2@X.
       
        """
        # Build the 4x4 system matrix A
        # Rows are derived from the cross product x_vector cross (P @ X_vector) = 0
        A = np.array([
            p1[0] * P1[2, :] - P1[0, :],
            p1[1] * P1[2, :] - P1[1, :],
            p2[0] * P2[2, :] - P2[0, :],
            p2[1] * P2[2, :] - P2[1, :]
        ])
        
        # Solve Ax = 0 via Singular Value Decomposition (SVD)
        # The solution is the unit eigenvector associated with the smallest eigenvalue
        _, _, Vh = np.linalg.svd(A)
        X_homo = Vh[-1]  # Last row of Vh
        
        # Normalize from homogeneous [x, y, z, w] to 3D [x/w, y/w, z/w]
        X_3d = X_homo[:3] / X_homo[3]
        return X_3d

# --- Validation Section ---
if __name__ == "__main__":
    # 1. Setup synthetic camera
    f = 1000
    cx, cy = 320, 240
    K = np.array([[f, 0, cx], [0, f, cy], [0, 0, 1]], dtype=float)
    
    # Camera 1 at origin, Camera 2 shifted slightly
    P1 = VisionUtils.get_projection_matrix(K, np.eye(3), np.zeros(3))
    P2 = VisionUtils.get_projection_matrix(K, np.eye(3), np.array([0.1, 0, 0]))
    
    # 2. Define a ground truth 3D point
    X_true = np.array([1.5, 2.0, 10.0])
    
    # 3. Project to both views
    p1 = VisionUtils.project_3d_to_2d(P1, X_true)
    p2 = VisionUtils.project_3d_to_2d(P2, X_true)
    
    # 4. Triangulate back to 3D
    X_reconstructed = VisionUtils.triangulate_dlt(p1, p2, P1, P2)
    
    # 5. Calculate Error
    error = np.linalg.norm(X_true - X_reconstructed)
    print(f"Ground Truth: {X_true}")
    print(f"Reconstructed: {X_reconstructed}")
    print(f"Triangulation Error: {error:.8f} mm")
    
    if error < 1e-3:
        print("✅ Validation Successful: Error is less than 1mm.")
    else:
        print("❌ Validation Failed: Error too high.")

Ground Truth: [ 1.5  2.  10. ]
Reconstructed: [ 1.5  2.  10. ]
Triangulation Error: 0.00000000 mm
✅ Validation Successful: Error is less than 1mm.
